In [1]:
import pandas as pd
from tqdm.auto import tqdm
from bulk_geocoding import geocode
from datetime import datetime
import duckdb
import numpy as np
from sqlalchemy import create_engine  # Make sure this is imported
import os

connection_source = duckdb.connect(database="../../dbt/database_name.duckdb")

/Users/raphaelcourivaud/anaconda3/envs/py310/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Replace the following with your database connection details
username = os.environ.get('ZLV_PROD_USERNAME')
password = os.environ.get('ZLV_PROD_PASSWORD')
host = os.environ.get('ZLV_PROD_HOST')
port = os.environ.get('ZLV_PROD_PORT')
database = os.environ.get('ZLV_PROD_DATABASE')

engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

# Synchronize duckdb with production

In [3]:
text_query = """
    ATTACH 'dbname=bbuyufrdks9xrduy8zbu user=uroqdghcooq5iv0tnmvq password=ngxngxqucboptqbqowol0mspq7kcoz host=bbuyufrdks9xrduy8zbu-postgresql.services.clever-cloud.com port=5387' AS zlv_replication_db (TYPE POSTGRES, READ_ONLY);
    CREATE OR REPLACE table production.owners AS (SELECT * FROM zlv_replication_db.public.owners );
    CREATE OR REPLACE table production.owners_housing AS (SELECT * FROM zlv_replication_db.public.owners_housing );
    CREATE OR REPLACE table production.housing AS (SELECT * FROM zlv_replication_db.public.fast_housing );
    CREATE OR REPLACE table production.ban_addresses AS (SELECT * FROM zlv_replication_db.public.ban_addresses );
"""

connection_source.execute(text_query).fetchall()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[(28761737,)]

In [4]:
text_query = """
    WITH owners_in_lovac_2024 AS (
        SELECT o.id, o.address_dgfip
        FROM production.owners o
        LEFT JOIN production.owners_housing oh ON oh.owner_id = o.id
        LEFT JOIN production.housing h ON h.id = oh.housing_id
        WHERE array_contains(h.data_file_years, 'lovac-2024')
    ),  addresses AS (
        SELECT o.id, unnest(o.address_dgfip) as address
        FROM owners_in_lovac_2024 o
        LEFT JOIN production.ban_addresses ba
            ON ba.ref_id = o.id AND ba.address_kind = 'Owner'
        WHERE ba.ref_id IS NULL AND o.address_dgfip IS NOT NULL
    ) SELECT id, string_agg(address, ', ') AS aggregated_address
    FROM addresses
    GROUP BY id;
"""

df = connection_source.execute(text_query).fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
# Define a mapping from original columns to the desired output columns
column_mapping = {
    'id': 'ref_id',
    'result_type': 'address_kind',
    'result_housenumber': 'house_number',
    'result_street': 'street',
    'result_postcode': 'postal_code',
    'result_city': 'city',
    'latitude': 'latitude',
    'longitude': 'longitude',
    'result_score': 'score',
    'result_id': 'ban_id', 
    'result_label': 'address'
}

# List the required columns in the desired order
required_columns = [
    'ref_id', 'address_kind', 'house_number', 
    'street', 'postal_code', 
    'city', 'latitude', 'longitude', 'score',
    'last_updated_at', 'address', 'ban_id'
]

chunk_size = 10000  # Example chunk size
SIZE = len(df)  # Total number of rows in the DataFrame

# Splitting the DataFrame into chunks


def process_geocoding(df, engine, table_name, output_path, chunk_size=10000):
    chunks = np.array_split(df, SIZE // chunk_size)

    for chunk in tqdm(chunks, total=SIZE // chunk_size):
        try:
            chunk_records = chunk.to_dict('records')
            geocoded_chunk = geocode(chunk_records, columns=['aggregated_address'])
            geocoded_df = pd.DataFrame(geocoded_chunk)
            geocoded_df.rename(columns=column_mapping, inplace=True)
            geocoded_df['last_updated_at'] = datetime.now().isoformat()
            geocoded_df['address_kind'] = 'Owner'

            geocoded_df = geocoded_df[required_columns]
            geocoded_df.replace('', None, inplace=True)  # Replace empty strings with None
            geocoded_df['latitude'] = geocoded_df['latitude'].astype(float, errors='ignore')  # Ensure type consistency
            geocoded_df['longitude'] = geocoded_df['longitude'].astype(float, errors='ignore')  # Ensure type consistency
            geocoded_df['score'] = geocoded_df['score'].astype(float, errors='ignore')  # Ensure type consistency
            geocoded_df = geocoded_df.dropna(subset=['address'])
            # Now try inserting the data into the database
            geocoded_df.to_csv(output_path, mode='a', index=False, header=not pd.io.common.file_exists(output_path))            
            geocoded_df.to_sql(table_name, con=engine, if_exists='append', index=False)
        except Exception as e:
            print(e)
            pass

In [7]:
process_geocoding(df=df, engine=engine, table_name='ban_addresses', output_path='ban_addresses.csv', chunk_size=chunk_size)

/Users/raphaelcourivaud/anaconda3/envs/py310/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/86 [00:00<?, ?it/s]

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "ban_addresses_pkey"
DETAIL:  Key (ref_id, address_kind)=(02bf2edf-d658-41d7-986e-fe5098954967, Owner) already exists.

[SQL: INSERT INTO ban_addresses (ref_id, address_kind, house_number, street, postal_code, city, latitude, longitude, score, last_updated_at, address, ban_id) VALUES (%(ref_id__0)s, %(address_kind__0)s, %(house_number__0)s, %(street__0)s, %(postal_code__0)s ... 233487 characters truncated ... )s, %(longitude__999)s, %(score__999)s, %(last_updated_at__999)s, %(address__999)s, %(ban_id__999)s)]
[parameters: {'longitude__0': 1.994112, 'ref_id__0': 'd5a66435-17e7-4f4a-a2f1-e5ea5d24911c', 'street__0': 'Le Coudonie', 'house_number__0': None, 'ban_id__0': '11139_tddosz', 'postal_code__0': '11240', 'city__0': 'Fenouillet-du-Razès', 'last_updated_at__0': '2024-10-17T10:21:44.278628', 'address_kind__0': 'Owner', 'latitude__0': 43.152365, 'address__0': 'Le Coudonie 11240 Fenouillet-du-Razès', 'score_

In [1]:
import pandas as pd

/Users/raphaelcourivaud/anaconda3/envs/py310/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df_ = pd.read_csv("Results_44_geocoded.csv")

In [7]:
df_["address_kind"] = "Owner"

In [8]:
df_.to_csv("Results_44_geocoded_updated.csv", index=False)

In [10]:
df.query("id = 'df7ac216-1ecf-48bb-b6d9-d1cbeb25ff88'")

NameError: name 'df' is not defined

In [11]:
df = pd.read_csv(input_path)


In [15]:
df.query("id == 'df7ac216-1ecf-48bb-b6d9-d1cbeb25ff88'")["aggregated_address"].values

array(["DOMAINE DE L ENFER 0000 CHE DE L'ENFER 63800 COURNON D AUVERGNE"],
      dtype=object)

In [18]:
df.merge(df_, left_on="id", right_on="ref_id", how="outer").query("ref_id.isnull()").to_csv("./to_refectch.csv", index=False)

In [3]:
df_to_store = pd.read_csv("Results_44_geocoded_updated.csv")

In [5]:
df_to_store.head()

,ref_id,address_kind,house_number,street,postal_code,city,latitude,longitude,score,last_updated_at,address,ban_id
0,22eb37a6-c4dc-48f4-b846-8e50b90b88c6,Owner,18,Avenue des Albères,66170.0,Millas,42.686897,2.691008,0.798896,2024-10-15T13:59:17.326149,"18 AV DES ALBERES, 66170 MILLAS",66108_0002_00018
1,d951b669-7727-4243-b60e-cea488b9d6ac,Owner,9,Rue Boilly,31300.0,Toulouse,43.602254,1.423946,0.724134,2024-10-15T13:59:17.326149,APPT 1 0009 RUE BOILLY 31300 TOULOUSE,31555_1232_00009
2,df7ac216-1ecf-48bb-b6d9-d1cbeb25ff88,Owner,NaN,chemin de l'Enfer,63800.0,Cournon-d'Auvergne,45.723468,3.176418,0.580380,2024-10-15T13:59:17.326149,DOMAINE DE L ENFER 0000 CHE DE L'ENFER 63800 C...,63124_0373
3,1f540d4e-ede3-4d21-b883-ea7c4f148fd3,Owner,NaN,NaN,11100.0,Narbonne,43.208169,3.004420,0.956976,2024-10-15T13:59:17.326149,DOMAINE DE PETIT BOUTES 11100 NARBONNE,11262_B213
4,5d903c34-619d-4125-8601-c48c4f26a9fa,Owner,121,Avenue de l'Espagne,13110.0,Port-de-Bouc,43.422393,5.003375,0.431606,2024-10-15T13:59:17.326149,121 CH DU CLAPAS LOT ESCAILLON LE VILLAGE 1311...,13077_0182_00121
